In [0]:
import tarfile
from google.colab import drive
# drive.mount('/content/drive/')
!fusermount -u drive
drive.mount('/content/drive')

In [0]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import random
import math as ma
import time
from statistics import mean 
from itertools import combinations
import csv
import codecs
import zipfile as zipfile
from sklearn.decomposition import PCA

In [0]:
# Generate data

paraN=64
agentN=10
dataN=200
sampleN=agentN*dataN
mu=np.zeros((1, sampleN))
Sigma=np.eye(sampleN)
np.random.seed(0)
M=np.random.multivariate_normal(mu[0],Sigma,paraN).T
e=np.random.normal(0, 1, sampleN)

# Normalize data
M =[ma.sqrt(int(paraN))* (np.sqrt(np.abs(LA.eig(np.outer(M[i], M[i]))[0]/ max(LA.eig(np.outer(M[i], M[i]))[0])))).dot(LA.eig(np.outer(M[i], M[i]))[1].T) for i in range(sampleN)]
M=np.real(M)
M=np.array(M)
#e=(2/(np.abs(e)).max(axis=0))*e
#e=np.zeros((sampleN, 1))
e=0.1*e/LA.norm(e)
x_star_nonnorm=np.random.random(size=paraN)-0.5
x_star_norm=300*x_star_nonnorm/LA.norm(x_star_nonnorm)
x_star=np.array(x_star_norm)
y=[M.dot(x_star.T)[i]+e[i] for i in range(sampleN)]
y=np.array(y)
y=y.reshape((sampleN, 1))
y_local=[y[i*dataN:(i+1)*dataN, :] for i in range(agentN)]
y_local=np.array(y_local)
M_local=[M[i*dataN:(i+1)*dataN, :] for i in range(agentN)] # generate groups of local data
M_local=np.array(M_local)
x_star=np.array([x_star for j in range(agentN)])

# Construct the graph
def generate_w(Idx, agentN):
    G=np.zeros([agentN,agentN])
    for i in Idx:
        G[i[0],i[1]] = 1 
    G=G+G.T
    d=G.sum(axis=1)
    # Construct the mixing matrix
    W=np.zeros([agentN,agentN])
    for i in Idx:
        W[i[0], i[1]] = 0.5/max(d[i[0]], d[i[1]])
    W=W+W.T
    diag=np.eye(agentN,agentN)
    for i in range(agentN):
        diag[i,i] = 1 - np.sum(W[i])
    W=W+diag
    return W


epsilon=0.45
p=(1+epsilon)*ma.log(agentN)/agentN
C=0

def erdos(N, p):
    A=np.random.rand(N,N)<p
    A=np.triu(A, 1)
    A=A+A.T
    Lap=np.diag(A*np.ones((N, 1)))-A
    V_L, Lambda_L=np.linalg.eig(Lap)
    if Lambda_L[2,2]<=1e-8:
        C=0
    else:
        C=1
    return A, C

def generate_conn_w(agentN, N, p):
    C=0
    while C==0:
        A1,C=erdos(N, p)
    d=A1.sum(axis=1)
    # Construct the mixing matrix
    W=np.zeros([agentN,agentN])
    A2=A1-np.diag(A1*np.ones((N,1)))
    Idx_n=np.argwhere(A1)
    for i in Idx_n:
        W[i[0], i[1]] = 0.5/max(d[i[0]], d[i[1]])
    diag=np.eye(agentN)
    for i in range(agentN):
        diag[i,i] = 1 - np.sum(W[i])
    W=W+diag
    return W

Idx=[]
for i in range(agentN):
    for j in range(i+1, agentN):
        elem=np.array([[i, j]])
        Idx=Idx+elem.tolist()

print(Idx)
l = [1, 2, 3, 4, 5]
fracx=4


def compute_gamma_e(w, i, frac):
    if frac==0:
        return 1/w[i]
    else:
        w_list=list(w)
        w_list.pop(i)
        w_inverse=np.asarray(list(combinations(w_list, frac)))
        w_inverse=[w_inverse[k]+w[i]/len(w_list) for k in range(w_inverse.shape[0])]
    #    print(w_inverse)
    #    w_inverse2=np.concatenate((w_inverse,w[i]*np.ones(w_inverse.shape[0]).T))
        w_inverse_sum=1/((np.sum(w_inverse, axis=1)))
        return np.sum(w_inverse_sum)



In [0]:
# Consensus - Directed DGD

agentN=10
x_star_init=np.array([10*np.random.random(size=paraN)-5 for j in range(agentN)])
x_star_init=np.array([[x_star_init[i][j] if abs(x_star_init[i][j])<5 else 5 for j in range(paraN)] for i in range(agentN)])
x_star_consensus_ave=np.sum(x_star_init, axis=0)/agentN
x_star_consensus=np.array([x_star_consensus_ave for j in range(agentN)])
p=0.75

z_init=np.vstack((x_star_init, np.zeros((agentN, paraN))))
z_star_consensus_ave=np.sum(x_star_init, axis=0)/(agentN)
z_star_consensus=[z_star_consensus_ave if k< agentN else np.zeros((1, paraN))[0] for k in range(2*agentN)]
#print(z_star_consensus)


N=5000
epsilon=0.01

x_a1_cons=x_star_init
y_a1_cons=np.ones((agentN, 1))
x_a1_hat=np.zeros((agentN, paraN))
x_a2_cons=x_star_init
#mu, sigma = 0, 0.0001


def directed_dp_con(N, sel_num, agentN, paraN, epsilon, sigma, mode):
    z_a1_cons=z_init
    loss_z_cons=[]
    for i in range(int(N)):
        z_a1_cons_col=np.zeros((agentN*2, paraN))
        for j in range(paraN):
            # Construct Matrix M
            # Mechanism 1
            A=np.random.random((agentN,agentN))
            B=np.random.random((agentN, agentN))
            A_new=np.array([A[k] for k in range(agentN)])
            B_new=np.array([B[k] for k in range(agentN)])
            # Mechanism 2
#            p=0.95
#            Wk=generate_conn_w(agentN, agentN, p)
#            A_new=np.array([Wk[k] for k in range(agentN)])
#            B_new=np.array([Wk[k] for k in range(agentN)])
            
            for m in range(agentN):
                w_list=list(range(agentN))
                w_list.pop(m)
                if sel_num=='full':
                    w_num=0
                elif sel_num=='none':
                    w_num=agentN-1
                else:
                    w_num=sel_num
                w_id=random.sample(w_list,w_num)

                A_new[m][w_id]=0
                B_new[m][w_id]=0

            A=np.array([A_new[k]/sum(A_new[k]) for k in range(agentN)]) # Make it row-stochastic            
            B=np.array([B_new.T[k]/sum(B_new.T[k]) for k in range(agentN)]) #Make it column-stochastic
            B=B.T
            s_idx=np.random.binomial(1, 0.9)
            if s_idx==1:
                A=np.eye(agentN)
                B=np.eye(agentN)
            M_up=np.hstack((A, epsilon*np.eye(agentN)))
            M_down=np.hstack((np.eye(agentN)-A, B-epsilon*np.eye(agentN)))
            M_new=np.vstack((M_up, M_down))
        #    M_1=np.array([M_new[k] for k in range(2*agentN)])
            s = np.random.normal(0, sigma, (agentN, 1))
#            z_a1_cons_col_new=np.array([z_a1_cons[k, j]+s[k, 0] if k<agentN else z_a1_cons[k, j]-s[k-agentN, 0] for k in range(agentN*2)])
#            z_a1_cons_col[:, j]=M_new.dot(z_a1_cons_col_new.T)
            z_a1_cons_col[:, j]=M_new.dot(z_a1_cons[:, j])
            if i==0 and mode=='dp':
                z_a1_cons_col_noise=np.array([s[k, 0] if k<agentN else -s[k-agentN, 0] for k in range(agentN*2) ])
                z_a1_cons_col[:, j] += z_a1_cons_col_noise
            

        #    z_a1_cons=M_new.dot(z_a1_cons)
        z_a1_cons=np.array([z_a1_cons_col[k] for k in range(2*agentN)])
        z_a1_ave=np.sum(z_a1_cons, axis=0)/(agentN)
        z_a1_consensus=np.array([z_a1_ave for j in range(agentN)])
        z_a1_consensus_xy=np.vstack((z_a1_consensus, np.zeros((agentN, paraN))))
        #    print(z_a1_cons.shape, z_a1_consensus.shape)
#        loss_z_cons.append(LA.norm(z_a1_cons-z_a1_consensus_xy))
        loss_z_cons.append(LA.norm(z_a1_cons-z_star_consensus))
        print(i)
    return loss_z_cons
    



In [0]:
# Run experiments with different parameters and plot

N=5000
#loss_z_cons_f=directed_dp_con(N, "full", agentN, paraN, 0.01, 1, 'f')
#loss_z_cons_r=directed_dp_con(N, "full", agentN, paraN, 0.01, 3, 'dp')
#loss_z_cons_r1=directed_dp_con(N, agentN-5, agentN, paraN, 0.01, 1, 'f')
#loss_z_cons_r2=directed_dp_con(N, agentN-5, agentN, paraN, 0.01, 3, 'dp')


start=1
y_axis = np.random.normal(loc=0.5, scale=0.4, size=1000)
y_axis = y_axis[(y_axis > 0) & (y_axis < 1)]
#plt.ylim((1e-10, 1))
y_axis.sort()    
plt.clf
#plt.ylim((1e-12, 100))
#plt.xlim((1, 2600))
#plt.xscale('log')
plt.yscale('log')
plt.grid()

plt.plot(loss_z_cons_f, color='r', linestyle="-", label ="Full Communication")
plt.plot(loss_z_cons_r, color='g', linestyle="-.", label ="Full Communication Privacy")
plt.plot(loss_z_cons_r1, color='y', linestyle="-.", label ="0.5 Communication ")
plt.plot(loss_z_cons_r2, color='b', linestyle="-", label ="0.5 Communication Privacy")

plt.xlabel('iteration t')
plt.ylabel('Residual')
# plt.legend()
plt.legend(bbox_to_anchor=(1.01, 0.70),loc='upper right')
plt.savefig('dp_consensus.eps',dpi=600,format='eps')